In [0]:
from pyspark.sql.functions import col

In [0]:
%run "../includes/common_functions"

In [0]:
latest_update = spark.sql("""select * 
                          from cars_catalog.default.metadata_table
                          where table_name = 'fact_sales' 
                          """).collect()[0][1]
latest_update

datetime.datetime(2025, 5, 7, 8, 20, 2, 865000)

In [0]:
fact_table = spark.read.format("delta") \
                        .load("abfss://gold@cardeprojectdl.dfs.core.windows.net/fact_sales") \
                        .filter(col("updated_at") == latest_update)

In [0]:
fact_table.display()

In [0]:
sql_create_table = """
    CREATE TABLE IF NOT EXISTS CARSALES_PROJECT.DW.fact_sales (
        Revenue NUMBER,
        Units_Sold NUMBER,
        RevPerUnit DOUBLE,
        dim_branch_key NUMBER,
        dim_dealer_key NUMBER,
        dim_date_key NUMBER,
        dim_model_key NUMBER,
        updated_at TIMESTAMP
    )

"""

In [0]:
sql_merging = """
            MERGE INTO CARSALES_PROJECT.DW.fact_sales AS tar
            USING CARSALES_PROJECT.EXTERNAL_STAGE.STG_FACT_SALES AS src
            ON tar.dim_model_key = src.dim_model_key and tar.dim_dealer_key = src.dim_dealer_key and tar.dim_branch_key = src.dim_branch_key and tar.dim_date_key = src.dim_date_key
            WHEN MATCHED THEN
                UPDATE SET tar.Revenue = src.Revenue,
                            tar.Units_Sold = src.Units_Sold,
                            tar.RevPerUnit = src.RevPerUnit,
                            tar.updated_at = src.updated_at

            WHEN NOT MATCHED THEN   
                INSERT (Revenue, Units_Sold, RevPerUnit, dim_branch_key, dim_dealer_key, dim_date_key, dim_model_key, updated_at) VALUES (src.Revenue, src.Units_Sold, src.RevPerUnit, src.dim_branch_key, src.dim_dealer_key, src.dim_date_key, src.dim_model_key,  src.updated_at)
"""

In [0]:
write_to_snowflake(fact_table,"STG_FACT_SALES",sql_create_table,sql_merging)